In [10]:
import pandas as pd 
import numpy as np
import pyblp
from sklearn.linear_model import LinearRegression

In [11]:
product_data = pd.read_csv('../data/market_integrates_1.csv')
print(product_data)

     market_ids  firm_ids  characteristic1  characteristic2    prices  \
0             1         1         5.417022        20.419195  2.606747   
1             1         2         5.720324        20.685220  2.380700   
2             1         3         5.000114        20.204452  2.576762   
3             1         4         5.302333        20.878117  2.452763   
4             1         5         5.146756        20.027388  2.696566   
..          ...       ...              ...              ...       ...   
995         100         6         5.092339        20.670468  2.559899   
996         100         7         5.186260        20.417305  2.736607   
997         100         8         5.345561        20.558690  2.523615   
998         100         9         5.396767        20.140387  2.527186   
999         100        10         5.538817        20.198101  2.600215   

     marginal_cost    shares   profits   markups  e_quantity    capital  \
0         2.000318  0.012468  0.007561  1.303166

In [12]:
# Getting the dataset estimation ready with at least a supply_side instrument/ demand_side instrument 
product_data_complete = product_data.rename(columns={'labor':'supply_instrument0', 'capital':'supply_instruments1', 'marginal_cost': 'supply_instrument2'})
print(product_data_complete)



     market_ids  firm_ids  characteristic1  characteristic2    prices  \
0             1         1         5.417022        20.419195  2.606747   
1             1         2         5.720324        20.685220  2.380700   
2             1         3         5.000114        20.204452  2.576762   
3             1         4         5.302333        20.878117  2.452763   
4             1         5         5.146756        20.027388  2.696566   
..          ...       ...              ...              ...       ...   
995         100         6         5.092339        20.670468  2.559899   
996         100         7         5.186260        20.417305  2.736607   
997         100         8         5.345561        20.558690  2.523615   
998         100         9         5.396767        20.140387  2.527186   
999         100        10         5.538817        20.198101  2.600215   

     supply_instrument2    shares   profits   markups  e_quantity  \
0              2.000318  0.012468  0.007561  1.303166 

In [13]:
# the endog regressor
X1_formulation = pyblp.Formulation('1 + prices + characteristic1 + characteristic2')

# the only regressor on which we want random coeff 
X2_formulation = pyblp.Formulation('0 +  prices')

product_formulations = (X1_formulation, X2_formulation)
print(product_formulations)

(1 + prices + characteristic1 + characteristic2, prices)


In [14]:
mc_integration = pyblp.Integration('monte_carlo', size=50, specification_options={'seed': 0})


In [15]:
pr_integration = pyblp.Integration('product', size=5)


In [16]:
mc_problem = pyblp.Problem(product_formulations, product_data_complete, integration=mc_integration)
print(mc_problem)

Initializing the problem ...
Initialized the problem after 00:00:00.

Dimensions:
 T    N     F    I     K1    K2    MD 
---  ----  ---  ----  ----  ----  ----
100  1000  10   5000   4     1     3  

Formulations:
       Column Indices:           0       1            2                3       
-----------------------------  ------  ------  ---------------  ---------------
 X1: Linear Characteristics      1     prices  characteristic1  characteristic2
X2: Nonlinear Characteristics  prices                                          
Dimensions:
 T    N     F    I     K1    K2    MD 
---  ----  ---  ----  ----  ----  ----
100  1000  10   5000   4     1     3  

Formulations:
       Column Indices:           0       1            2                3       
-----------------------------  ------  ------  ---------------  ---------------
 X1: Linear Characteristics      1     prices  characteristic1  characteristic2
X2: Nonlinear Characteristics  prices                                          


In [17]:
bfgs = pyblp.Optimization('bfgs', {'gtol': 1e-4})


In [18]:
results1 = mc_problem.solve(sigma=np.ones((1, 1)), optimization=bfgs)


Solving the problem ...

Nonlinear Coefficient Initial Values:
Sigma:     prices    
------  -------------
prices  +1.000000E+00
Starting optimization ...

GMM   Optimization   Objective   Fixed Point  Contraction  Clipped    Objective      Objective      Gradient                  
Step   Iterations   Evaluations  Iterations   Evaluations  Shares       Value       Improvement       Norm           Theta    
----  ------------  -----------  -----------  -----------  -------  -------------  -------------  -------------  -------------
 1         0             1           483         1549         0     +8.899467E-18                 +3.104968E-08  +1.000000E+00

Optimization completed after 00:00:00.
Computing the Hessian and and updating the weighting matrix ...
Computed results after 00:00:01.

Problem Results Summary:
GMM     Objective      Gradient                    Clipped  Weighting Matrix
Step      Value          Norm          Hessian     Shares   Condition Number
----  -------------